### You will learn:
- Split a pnadas object into pieces using one or more keys (in the forms of functions, arrays, or DataFrame column names)
- Calculate group summary statistics, like count, mean, or standard deviation, or a user-defined function.
- Within-group trandformation or other manpulations, like normalization, linear regression, rank , or subset selection.
- Pivot tables and cross tabulation
- Quantile analysis and other statistical group analysis.

# 10.1 GroupBy Mechanics
#### Split-Apply-Combine
Keys do not have to be all of the same type.
- A list or array of values that is the same length as the axis being grouped.
- A value indicating a column name in a DataFrame.
- A dict or Series giving a correspondence b/w the values on the axis being grouped and the group names.
- A function to be involved on the axis index or the individual labels in the index.


In [2]:
import pandas as pd
import numpy as np

#### Groupby using series.

In [4]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'], 
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randn(5),
                  'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-1.618715,-1.962461
1,a,two,0.286862,-0.972393
2,b,one,-0.986526,-0.320113
3,b,two,0.486519,-0.877098
4,a,one,0.359921,-0.940876


In [5]:
# computing mean of 'data1' by labels from 'key1'
grouped = df['data1'].groupby(df['key1'])
grouped   # Groupby object

In [6]:
grouped.mean()

key1
a   -0.323977
b   -0.250004
Name: data1, dtype: float64

Grouped the data using two keys, resulting in a hierarchia index, **consisting of unique pairs of keys**

In [8]:
mean = df['data1'].groupby([df['key1'], df['key2']]).mean()
mean

key1  key2
a     one    -0.629397
      two     0.286862
b     one    -0.986526
      two     0.486519
Name: data1, dtype: float64

In [9]:
mean.unstack()

key2,one,two
key1,,
a,-0.629397,0.286862
b,-0.986526,0.486519


#### Group keys could be any arrays of the right length.

In [10]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.286862
            2006   -0.986526
Ohio        2005   -0.566098
            2006    0.359921
Name: data1, dtype: float64

In [13]:
# No 'key2' column; since it not numeric data, it is said to be a nuisance column.
# therefore excluded from the result.
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.323977,-1.291910
b,-0.250004,-0.598605


In [15]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.629397 -1.451669
     two   0.286862 -0.972393
b    one  -0.986526 -0.320113
     two   0.486519 -0.877098

#### `size` GroupBy method returns a Series containing group sizes.

In [16]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## Iterating Over Groups
A sequence of 2-tuples containing the group name along with the chunk of data.

In [17]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -1.618715 -1.962461
1    a  two  0.286862 -0.972393
4    a  one  0.359921 -0.940876
b
  key1 key2     data1     data2
2    b  one -0.986526 -0.320113
3    b  two  0.486519 -0.877098


In [18]:
# In case of multiple groupby keys, the first element in the tuple will bea tuple of key values
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -1.618715 -1.962461
4    a  one  0.359921 -0.940876
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.286862 -0.972393
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.986526 -0.320113
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.486519 -0.877098


In [21]:
# computing a dict of the data pieces
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.986526,-0.320113
3,b,two,0.486519,-0.877098


In [23]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

#### Grouping columns of df by their dtype 

In [24]:
grouped = df.groupby(df.dtypes, axis=1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -1.618715 -1.962461
1  0.286862 -0.972393
2 -0.986526 -0.320113
3  0.486519 -0.877098
4  0.359921 -0.940876
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## Selecting a column or Subset of columns
Indexing a groupby object created from a dataframe with a column name or array of column names has the effect of column subsetting for aggregation.

i.e.

- `df.groupby('key1')['data1']` is same as `df['data1'].groupby('key1')`
- `df.groupby('key1')[['data1']]` is same as `df[['data1']].groupby('key1')`

### Use `df[['data1', 'data2']].groupby('key1')` or `df.groupby('key1')[['data1', 'data2']]` 
Note:
1. `df['data1']` return series.
2. `df[['data1']]` return DataFrame

In [29]:
# Return Grouped DataFrame
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.451669
     two  -0.972393
b    one  -0.320113
     two  -0.877098

In [30]:
# return a Series
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one    -1.451669
      two    -0.972393
b     one    -0.320113
      two    -0.877098
Name: data2, dtype: float64

## Grouping with Dicts and Series

In [31]:
people = pd.DataFrame(np.random.randn(5, 5), 
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # add a few NA values
people

,a,b,c,d,e
Joe,0.491639,0.005457,-0.565988,-1.133717,0.840715
Steve,2.225288,0.292086,1.119945,-0.187211,-0.270843
Wes,-0.123936,NaN,NaN,-1.157277,-0.853124
Jim,-0.218272,1.586095,-0.029932,0.573023,-0.608957
Travis,0.324449,0.972163,-0.132364,1.578709,-0.298674


#### Group correspondence for the columns and want to sum together the columns by group

In [34]:
mapping = {
    'a': 'red',
    'b': 'red',
    'c': 'blue',
    'd': 'blue',
    'e': 'red',
    'f': 'orange',  # its OK if you have unused key
}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-1.699705,1.337811
Steve,0.932735,2.246531
Wes,-1.157277,-0.977060
Jim,0.543090,0.758866
Travis,1.446346,0.997938


In [35]:
# Doing the grouping WITH pd.Series
mapping_series = pd.Series(mapping)
mapping_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [38]:
people.groupby(mapping_series, axis=1).sum()

,blue,red
Joe,-1.699705,1.337811
Steve,0.932735,2.246531
Wes,-1.157277,-0.977060
Jim,0.543090,0.758866
Travis,1.446346,0.997938


## Grouping with Functions
Any function passed as a group key will be called once per index value, with the return values being used as the group names.

In [39]:
# grouping by the length of anmes;
# return value: len(string)
people.groupby(len).sum()

,a,b,c,d,e
3,0.149431,1.591552,-0.595920,-1.717971,-0.621365
5,2.225288,0.292086,1.119945,-0.187211,-0.270843
6,0.324449,0.972163,-0.132364,1.578709,-0.298674


#### mixing functions with array, dicts, or Series is not a problem as everything gets converted to arrays internally

In [46]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.123936  0.005457 -0.565988 -1.157277 -0.853124
  two -0.218272  1.586095 -0.029932  0.573023 -0.608957
5 one  2.225288  0.292086  1.119945 -0.187211 -0.270843
6 two  0.324449  0.972163 -0.132364  1.578709 -0.298674

## Grouping by Index Levels
#### Ability to aggregate using one of the levels of an axis index in hierarchically indexed datasets.

In [47]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0     -0.002693 -1.723659 -0.822230  0.013900 -1.249483
1     -0.369946  1.353184  0.970262  0.697615  0.062247
2     -0.370733 -0.441418 -0.017596 -0.201255  1.708417
3      0.959604 -0.021401 -1.214982  0.335354  0.665248

#### To group by level, pass the number or name using the level keyword.

In [48]:
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 10.2 Data Aggregation
**Aggregation**: refers to any data transformation that produces scalar values from arrays.

In [49]:
df

,key1,key2,data1,data2
0,a,one,-1.618715,-1.962461
1,a,two,0.286862,-0.972393
2,b,one,-0.986526,-0.320113
3,b,two,0.486519,-0.877098
4,a,one,0.359921,-0.940876


In [50]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.345309
b    0.339214
Name: data1, dtype: float64

#### To use your own aggregation functions, pass any function that aggregates an array to `aggregate` or `agg` method. 

In [51]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.978636,1.021585
b,1.473045,0.556985


In [52]:
# some methods like describe also work, but stictly speaking they are not aggregations
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.323977  1.121871 -1.618715 -0.665926  0.286862  0.323392   
b      2.0 -0.250004  1.041600 -0.986526 -0.618265 -0.250004  0.118258   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.359921   3.0 -1.291910  0.580928 -1.962461 -1.467427 -0.972393   
b     0.486519   2.0 -0.598605  0.393848 -0.877098 -0.737851 -0.598605   

                          
           75%       max  
key1                      
a    -0.956635 -0.940876  
b    -0.459359 -0.320113

## Column-Wise and Multiple Function Application

In [54]:
tips = pd.read_csv('datasets/tips.csv')
# add tip `percentage to total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[: 6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


#### for descriptive analysis, you can pass the name of function as a string also.

In [55]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']

grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [62]:
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

#### If you pass a list of functions or function names instead, you get back a DataFrame with column names taken from the functions

In [64]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

#### (col_name, function) to agg method

In [66]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

### For DataFrames
Suppose we wanteed to compute the same three statistics for the `tip_pct` and `total_bill`

In [69]:
# you would get the same as aggregating (applying the function to each column) and then using concat on index
functions = ['count', 'mean', 'max']
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [70]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [71]:
# passing a list of tuples to column names for the result of agg functions

ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

#### To apply different functions to one or more of the columns; pass a dict to agg that contains a mapping of column names to any of the function specification listed so far.

In [73]:
grouped.agg({'tip': np.max, 'size': sum})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [74]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

## Returning Aggregated Data Without Row Indexes on which they were grouped
*`as_index`*`= False` to `groupby`.

Although you can use `reset_index`; but that adds additional computational overhead.

In [75]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [76]:
tips.groupby(['day', 'smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

# 10.3 Apply: General split-apply-combine
Suppose you wanted to select top five `tip_pct` values by group.

In [81]:
# First, write  function that selects the rows
# with the largest values in a particular column

def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column, ascending=False)[:n]
#   return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


### Up next...
The `top` function is called  on each row group from the DataFrame, and then the results are glued together using `pandas.concat`, labeling the pieces with the group names.

In [82]:
# Then groupping by 'smoker' and applying top function we just wrote.
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

If you pass a function to apply that takes other arguments or keywords, you can pass these after the function.

In [87]:
tips.groupby(['smoker', 'day']).apply(top, n=2, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
            91        22.49   3.50     No   Fri  Dinner     2  0.155625
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
            59        48.27   6.73     No   Sat  Dinner     4  0.139424
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
            112       38.07   4.00     No   Sun  Dinner     3  0.105070
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
            85        34.83   5.17     No  Thur   Lunch     4  0.148435
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
            90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
            102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
            184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982
            83        32.68   5.00    Yes  Thur   Lunch     2  0.152999

In [89]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [90]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

Inside `GroupBy` when you invoke a method like `describe`, it actually just a shortcut for:

In [91]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

## Suppressing the Group Keys
*`group_keys`*`= False`

Disble a hierarchical index formed from the group keys along with the indexes of each piece of the original object.

In [102]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


## Quantile and Bucket Analysis
The `Categorical` object returned by `cut` can be passed directly to groupby. SO we could compute a set of statistics for the `data2` column like so.

In [104]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quantiles = pd.cut(frame.data1, 4)
quantiles[:10]

0     (-1.246, 0.636]
1    (-3.136, -1.246]
2     (-1.246, 0.636]
3     (-1.246, 0.636]
4     (-1.246, 0.636]
5     (-1.246, 0.636]
6     (-1.246, 0.636]
7    (-3.136, -1.246]
8     (-1.246, 0.636]
9     (-1.246, 0.636]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.136, -1.246] < (-1.246, 0.636] < (0.636, 2.518] < (2.518, 4.4]]

In [105]:
def get_stats(group):
    return {'min': group.min(),
            'max': group.max(),
            'count': group.count(),
            'mean': group.mean()}
grouped = frame.data2.groupby(quantiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.136, -1.246]",-2.404693,2.459358,109.0,-0.175396
"(-1.246, 0.636]",-3.063710,3.068733,642.0,0.006965
"(0.636, 2.518]",-2.434410,2.257137,242.0,-0.111203
"(2.518, 4.4]",-1.781487,1.739974,7.0,-0.097850


These were equal length buckets; to compute equal size buckets based on sample quantiles, use `qcut`.

*`labels`*`= False` to just get quantile number

In [108]:
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.404693,1.991241,100.0,-0.219474
1,-2.270604,2.459358,100.0,0.123614
2,-3.063710,2.629895,100.0,0.146431
3,-2.456223,2.521601,100.0,-0.111213
4,-2.583156,3.028608,100.0,-0.088044
5,-2.426188,2.723629,100.0,0.004991
6,-1.911143,2.397772,100.0,0.065319
7,-2.239726,3.068733,100.0,-0.144781
8,-2.434410,2.231779,100.0,-0.054526


## Example: Filling Missing Values with Group-Specific Values
Usually we do it like this.

In [109]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.429661
2         NaN
3   -0.057757
4         NaN
5   -2.147924
dtype: float64

In [110]:
s.fillna(s.mean())

0   -0.878447
1   -0.429661
2   -0.878447
3   -0.057757
4   -0.878447
5   -2.147924
dtype: float64

### Fill value that vary by group.
One way to do is to group the data and use `apply` with a function that calls fillna on each data chunk.

In [111]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio          0.584645
New York     -0.472769
Vermont      -2.336430
Florida       0.471715
Oregon        0.263965
Nevada        1.327362
California    0.644738
Idaho        -0.931272
dtype: float64

In [114]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.584645
New York     -0.472769
Vermont            NaN
Florida       0.471715
Oregon        0.263965
Nevada             NaN
California    0.644738
Idaho              NaN
dtype: float64

In [115]:
# These calues are going to be filled in their respective group
data.groupby(group_key).mean()

East    0.194530
West    0.454352
dtype: float64

In [116]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.584645
New York     -0.472769
Vermont       0.194530
Florida       0.471715
Oregon        0.263965
Nevada        0.454352
California    0.644738
Idaho         0.454352
dtype: float64

### In another case, you might have predefined fill values that vary by group.
#### Use dict indicating group name as key and value you want to fill.
Groups have a `name` attribut set internally, so use that.

In [117]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

Ohio          0.584645
New York     -0.472769
Vermont       0.500000
Florida       0.471715
Oregon        0.263965
Nevada       -1.000000
California    0.644738
Idaho        -1.000000
dtype: float64

## Example: Random Sampling and Permutation

In [133]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S','C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']

cards = []
for suit in suits:
    # (str(num) + suit) for num in base_names
    cards.extend(str(name) + suit for name in base_names)
    
deck = pd.Series(card_val, index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

Drawing a hand of five cards from the deck could be written as

In [135]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

4C     4
KC    10
AS     1
QS    10
5S     5
dtype: int64

Drawing two random cards from each suit. Because the suit is the last character of each card name, we can group based on thos and use `apply`.

In [136]:
suit_letter = lambda card: card[-1] # last letter is suit

deck.groupby(suit_letter).apply(draw, n=2)

C  JC    10
   AC     1
D  4D     4
   QD    10
H  5H     5
   4H     4
S  JS    10
   KS    10
dtype: int64

Alternatively, we could write:

In [140]:
deck.groupby(suit_letter, group_keys=False).apply(draw, n=2)

4C      4
6C      6
9D      9
QD     10
QH     10
7H      7
5S      5
10S    10
dtype: int64

## Example: Group Weighted Average and Correlation

In [141]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.randn(8)})
df

,category,data,weights
0,a,0.547774,-0.264041
1,a,0.883264,-1.058727
2,a,0.733131,-0.092801
3,a,-0.650150,-0.243763
4,b,1.436115,-0.513475
5,b,-0.332067,0.958031
6,b,-0.860652,-0.095223
7,b,-0.609704,-0.790233


#### Then group weighted average by `category` would be:

In [142]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

df.groupby('category').apply(get_wavg)

category
a    0.596217
b    1.115395
dtype: float64

### Another example: Financial Dataset

In [143]:
close_px = pd.read_csv('datasets/stock_px_2.csv', parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [145]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


#### Computing a DataFrame consisting of the yearly correlations of daily returns (computed from percent changes) with SPX
One way to do this, we first create a function that computes the **pairwise correlation** of each column with `SPX` Column.

In [146]:
# spx_corr get every row in groups
spx_corr = lambda x: x.corrwith(x['SPX'])

Then compute **percent changes** on `close_px` using `pct_change`

In [147]:
rets = close_px.pct_change().dropna()

Lastly we **group percent changes by year**, which can be extracted from each row with a on-line function that returns the `year` attribute of each `datetime` label.

In [165]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)

rets.groupby(get_year).apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


#### Inter-column correlations
Annual Correlations between Apple and Microsoft

In [149]:
rets.groupby(get_year).apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression
As long as function returns a **pandas object** or **scalar value**, you can use `groupby` to perform more complex group-wise statistical anlysis.

In [166]:
import statsmodels.api as sm

def regress(date, yvar, xvars):
    Y = data[yvar]
    X = data[xvar]
    X['itercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

KeyError: 'AAPL'

# 10.4 Pivot Tables and Cross Tabulation
Pivot Table aggregates a table of data by one or more keys, arranging the data in a rectangle with some of the group keys along the rows and some along the columns.
- `pivot_table` can add partial totals, also known as *margins*

Suppose you wanted to compute a table of group means (the default `pivot_table` aggregation type) arranged by `day` and `smoker` on the rows:

In [171]:
tips[:2]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542


In [167]:
# this could have been produced by groupy directly
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

Now, suppose we want to aggregate only `tip_pct` and `size`, and additionally group by `time`. Putting `smoker` in table columns and `day` in rows.

In [168]:
tips.pivot_table(['tip_pct', 'size'], columns='smoker', index=['time', 'day'])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

### Include Partial totals by passing *`margins`*`= True`.

#### This has the effect of adding `All` row   and column labels, with corresponding values being the group statisitcs for all the data within a single tier.

In [173]:
# Here all means without taking into account smoker versus non-smoker (the All columns)
# OR
# any of the two levels of grouping on the rows (the All rows)
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

### To use different function, pass it to `aggfunc`.
'`count`' or `len` will give you cross-tabulation (count or frequency) of group sizes.

In [174]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

### If some combinations are empty, pass a `fill_value`.

In [177]:
tips.pivot_table(values='tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

## Cross-Tabulations: Crosstab
#### *Crosstab* is a special case of a pivot table that computes group frequencies.

Ohio          0.584645
New York     -0.472769
Vermont            NaN
Florida       0.471715
Oregon        0.263965
Nevada             NaN
California    0.644738
Idaho              NaN
dtype: float64